In [66]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from sklearn.metrics import mean_squared_error

def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [67]:
import db_fn as db

# Load the dataset
#df = pd.read_csv('../load_2023.csv', usecols=['date', 'load'])
df = pd.DataFrame(columns=['date','load'])
df = db.get_test()

df['date'] = pd.to_datetime(df['date'])
df = df.set_index(['date'])
print(df)

# Normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(df)

# Split into train and test sets
train_size = int(len(dataset) * 0.7)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size, :], dataset[train_size:len(dataset), :]

# reshape into X=t and Y=t+1
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

# Create and fit the LSTM network
model = Sequential()
model.add(LSTM(200, input_shape=(1, look_back)))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')

with tf.device("/device:GPU:0"):
    model.fit(trainX, trainY, epochs=50, batch_size=30, verbose=2)


connection Success!
                      load
date                      
2023-01-25 14:00:00  394.8
2023-01-25 15:00:00  390.6
2023-01-25 16:00:00  385.5
2023-01-25 17:00:00  364.2
2023-01-25 18:00:00  324.5
...                    ...
2023-07-25 09:00:00  377.1
2023-07-25 10:00:00  389.0
2023-07-25 11:00:00  391.6
2023-07-25 12:00:00  390.6
2023-07-25 13:00:00  395.8

[4344 rows x 1 columns]
Epoch 1/50
102/102 - 2s - loss: 0.0932
Epoch 2/50
102/102 - 0s - loss: 0.0185
Epoch 3/50
102/102 - 0s - loss: 0.0097
Epoch 4/50
102/102 - 0s - loss: 0.0092
Epoch 5/50
102/102 - 0s - loss: 0.0092
Epoch 6/50
102/102 - 0s - loss: 0.0092
Epoch 7/50
102/102 - 0s - loss: 0.0092
Epoch 8/50
102/102 - 0s - loss: 0.0092
Epoch 9/50
102/102 - 0s - loss: 0.0092
Epoch 10/50
102/102 - 0s - loss: 0.0091
Epoch 11/50
102/102 - 0s - loss: 0.0092
Epoch 12/50
102/102 - 0s - loss: 0.0092
Epoch 13/50
102/102 - 0s - loss: 0.0092
Epoch 14/50
102/102 - 0s - loss: 0.0092
Epoch 15/50
102/102 - 0s - loss: 0.0092
Epoch 16/50
1

In [77]:
def predict_load(model, end_date_str, df, scaler):
    end_date = pd.to_datetime(end_date_str, format='%Y-%m-%d %H')

    # Prepare the sequence of inputs for the model
    inputs = df.loc[:end_date]
    inputs = scaler.transform(inputs)

    # Make the predictions
    inputs = np.reshape(inputs, (inputs.shape[0], 1, inputs.shape[1]))
    predicted_load = model.predict(inputs)
    predicted_load = scaler.inverse_transform(predicted_load)

    # Prepare the output DataFrame
    dates = pd.date_range(df.index[0], end_date, freq='H')[:len(predicted_load)]
    output = pd.DataFrame(data={
        'date': dates,
        'load': predicted_load.flatten()
    })

    return output

end_date_str = '2023-07-30 23'  # Replace with your desired end date

predict_df = predict_load(model, end_date_str, df, scaler)
print(predict_df)

                    date        load
0    2023-01-25 14:00:00  387.294281
1    2023-01-25 15:00:00  383.639862
2    2023-01-25 16:00:00  379.187134
3    2023-01-25 17:00:00  360.367310
4    2023-01-25 18:00:00  323.855682
...                  ...         ...
4339 2023-07-25 09:00:00  371.847107
4340 2023-07-25 10:00:00  382.299927
4341 2023-07-25 11:00:00  384.555237
4342 2023-07-25 12:00:00  383.639862
4343 2023-07-25 13:00:00  388.134979

[4344 rows x 2 columns]


In [ ]:
# Make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

# Invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = np.squeeze(trainY)
trainY = scaler.inverse_transform([trainY])
testY = np.squeeze(testY)
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

print(testPredict)


from datetime import datetime

# The date we want to predict till
predict_date = '2023-07-30 23'
predict_date = datetime.strptime(predict_date, "%Y-%m-%d %H")

# Get the data up to the target date
predict_data = df[df.index <= predict_date]

# Normalize the data
predict_data_normalized = scaler.transform(predict_data)

# Create the dataset for the LSTM
predictX, _ = create_dataset(predict_data_normalized, look_back)
predictX = np.reshape(predictX, (predictX.shape[0], 1, predictX.shape[1]))

# Make prediction
prediction = model.predict(np.array([predictX[-1]]))  # Take the last sample

# Invert the prediction
prediction = scaler.inverse_transform(prediction)
print(f'The predicted load at {predict_date} is {prediction[0][0]}')


print(prediction)
